#### LangChain Essentials Course

# LangSmith Starter

LangSmith is a built-in observability service and platform that integrates _very easily_ with LangChain. You don't _need_ to use LangSmith for this course, but it can be very helpful in understanding what is happening, _and_ we recommend using it beyond this course for general development with LangChain — with all of that in mind we would recommend spending a little bit of time to get familiar with LangSmith.

## Setting up LangSmith

LangSmith does require an API key, but it comes with a generous free tier. You can sign up an account and get your API key [here](https://smith.langchain.com).

When using LangSmith, we need to setup our environment variables _and_ provide our API key, like so:

In [1]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-langsmith-openai"

In most cases, this is all we need to start seeing logs and traces in the [LangSmith UI](https://smith.langchain.com). By default, LangChain will trace LLM calls, chains, etc. We'll take a look at a quick example of this below.

## Default Tracing

As mentioned, LangSmith traces a lot of data without us needing to do anything. Let's see how that looks. We'll start by initializing our LLM. Again, this will need an [API key](https://platform.openai.com/api-keys).

In [2]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

Let's invoke our LLM and then see what happens in the LangSmith UI.

In [4]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-04a75541-53a4-4d38-b7f9-1be2b4a15031-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

This is settup for the non-langchain function.

In [7]:
import openai

client = openai

Now we want to use a non langchain related function, and these are not automatically traced by langsmith, so instead, we have to add the traceable decorator.

In [12]:
from langsmith import traceable

@traceable
def generate_response(question: str):

    complete_messages = [
        {"role": "system", "content": "You are a happy assistant"},
        {"role": "user", "content": question}
    ]

    return client.chat.completions.create(
        model = openai_model,
        messages = complete_messages,
        temperature = 0.1
    )

In [13]:
generate_response("How are you today?")

ChatCompletion(id='chatcmpl-AkCKHY4A66FAYToGl9lcN7YOVGALe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm feeling great, thank you! How about you? How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735573833, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=19, prompt_tokens=21, total_tokens=40, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

We can also add data into the traceable by editing its paramaters, this includes metadata, and what we will change, the function name.

In [14]:
from langsmith import traceable

@traceable(name="OpenAI Response")
def generate_second_response(question: str):

    complete_messages = [
        {"role": "system", "content": "You are a happy assistant"},
        {"role": "user", "content": question}
    ]

    return client.chat.completions.create(
        model = openai_model,
        messages = complete_messages,
        temperature = 0.1
    )

In [15]:
generate_second_response("How are you today?")

ChatCompletion(id='chatcmpl-AkCUBhB9ZPITBWIjGrPcbtXMWgfuZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm feeling great, thank you! How about you? How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735574447, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=19, prompt_tokens=21, total_tokens=40, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

Now you can view all of this over at the LangSmith website, under the tracing projects.